In [21]:
import os

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import Flow, InstalledAppFlow
from googleapiclient.discovery import build

def CreateService(clientSecretFile, apiName, apiVersion, scopes):
    print(clientSecretFile, apiName, apiVersion, scopes, sep='-')
    CLIENT_SECRET_FILE = clientSecretFile
    API_SERVICE_NAME = apiName
    API_VERSION = apiVersion

    creds = None

    credFile = f'token_{API_SERVICE_NAME}_{API_VERSION}.json'

    if os.path.exists(credFile):
        creds = Credentials.from_authorized_user_file(credFile, scopes)

    # 유효한 credential이 없다면 login하도록 한다.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRET_FILE, scopes)
            creds = flow.run_local_server(port=0)
        # 성공하면 기록하도록 한다.
        with open(credFile, 'w') as token:
            token.write(creds.to_json())

    # 서비스를 생성한다.
    try:
        service = build(apiName, apiVersion, credentials=creds, static_discovery=False)
        return service

    except Exception as e:
        print('Exception: ', e)

    return None
    

API_NAME = 'photoslibrary'
API_VERSION = 'v1'
CLIENT_SECRET_FILE = 'C:/Users/user/google-photo/client_secret.json'
SCOPES = ['https://www.googleapis.com/auth/photoslibrary',
          'https://www.googleapis.com/auth/photoslibrary.sharing']

service = CreateService(CLIENT_SECRET_FILE, API_NAME, API_VERSION, SCOPES)

#미디어 항목을 사용자의 라이브러리나 앨범에 업로드하려면 photoslibrary.appendonly또는 photoslibrary범위가 필요합니다.

C:/Users/user/google-photo/client_secret.json-photoslibrary-v1-['https://www.googleapis.com/auth/photoslibrary', 'https://www.googleapis.com/auth/photoslibrary.sharing']


In [34]:
#구글 포토 API 사용해보기

import requests
import json
import warnings

warnings.filterwarnings('ignore')

#service.albums().list().execute() 


upload_url= 'https://photoslibrary.googleapis.com/v1/uploads'

token = json.load(open('token_photoslibrary_v1.json','rb'))

headers= {
    'Authorization':'Bearer '+ token['token'],
    'Content-type':'application/octet-stream',
    'X-Goog-Upload-Protocol':'raw',
    'X-Goog-Upload-File-Name': "totoro name.jpg"
    
}

image_file = "C:/Users/user/Desktop/mail.naver.jpg"
filename = 'mail.naver'  #파일명

img = open(image_file,'rb').read()


response = requests.post(upload_url, data=img, headers = headers, verify=False)


request_body ={
    'newMediaItems':
    [
        {
            'description': filename,
            'simpleMediaItem': 
            {
                'uploadToken': response.content.decode('utf-8')
            }
        }
    ] 

}


upload_response = service.mediaItems().batchCreate(body=request_body).execute()


In [35]:
print(upload_response) #업로드 확인 완료

{'newMediaItemResults': [{'uploadToken': 'CAISiQMA9BP/VQa69GxlYkZxJCNfdZCatHL77dO0XM+uMepvTG5fPnaWhqfCiAVt1UGlP3fFZAq+sowmLKnfgWOKQOVIckV9O/n4lqeLcM6HHatw7drIoAjaKD5LBLFlltA3Km+cnnn6R6p8iY5tJKlgXu8lCP05qNCBIOyXE7qegLnxPDu76mqBm9iKJmkMOtgc/N8LoAt+Uul+TlxNutgjVjOzlVGSZkSjTymA7Ll/xSiqqXwYb0UOQOLNGNzl8OCf0b9VN/KLH646lSh5I2gqHFefVBP38kyvqJqpOjuFSJw/McNvMyyZWVQNu3yp1M5q3kHi8HEz01D7uiu6CEtBajiA4QmomhXvrGhJ8op+sYNA+FWsgMPtuFgpGyCesYxLZP/A5SmGCR9ZYLfJT5xhA4h0yzJJ20Bv522k1ZfLS/SJ14ziZITMXCMO9xV7UCVzFc36g/WNlhXKPu8mGKbqOoQCEpTNdD0gu+pvT9B2Br95IYcarehGWFGExkmEb1S+lbMuXhAD8HjDouc', 'status': {'message': 'Success'}, 'mediaItem': {'id': 'AI9aVGEZHB4RFxTwMRaVMmfJOc0QMhRKNM51ZRMsAdbyt6MvMKZ5t9CvdLqA-09abxzlx3WRZELnYV9-w_RIMmqAc9RKrk4pHQ', 'description': 'mail.naver', 'productUrl': 'https://photos.google.com/lr/photo/AI9aVGEZHB4RFxTwMRaVMmfJOc0QMhRKNM51ZRMsAdbyt6MvMKZ5t9CvdLqA-09abxzlx3WRZELnYV9-w_RIMmqAc9RKrk4pHQ', 'mimeType': 'image/jpeg', 'mediaMetadata': {'creationTime': '2022-10-15T00:28:56Z', 'wid